# Sequence Analysis with Python

Contact: Veli Mäkinen veli.makinen@helsinki.fi

The following assignments introduce applications of hashing with ```dict()``` primitive of Python. While doing so, a rudimentary introduction to biological sequences is given. 
This framework is then enhanced with probabilities, leading to routines to generate random sequences under some constraints, including a general concept of *Markov-chains*. All these components illustrate the usage of ```dict()```, but at the same time introduce some other computational routines to efficiently deal with probabilities.   
The function ```collections.defaultdict``` can be useful.

Below are some "suggested" imports. Feel free to use and modify these, or not. Generally it's good practice to keep most or all imports in one place. Typically very close to the start of notebooks.

In [ ]:
from collections import defaultdict
from itertools import product

import numpy as np
import pandas as pd
import random

The automated TMC tests do not test cell outputs. These are intended to be evaluated in the peer reviews. So it is still be a good idea to make the outputs as clear and informative as possible.

To keep TMC tests running as well as possible it is recommended to keep global variable assignments in the notebook to a minimum to avoid potential name clashes and confusion. Additionally you should keep all actual code exection in main guards to keep the test running smoothly. If you run [check_sequence.py](https://raw.githubusercontent.com/saskeli/data-analysis-with-python-summer-2019/master/check_outputs.py) in the `part07-e01_sequence_analysis` folder, the script should finish very quickly and optimally produce no output.

If you download data from the internet during execution (codon usage table), the parts where downloading is done should not work if you decide to submit to the tmc server. Local tests should work fine.

## DNA and RNA

A DNA molecule consist, in principle, of a chain of smaller molecules. These smaller molecules have some common basic components (bases) that repeat. For our purposes it is sufficient to know that these bases are nucleotides adenine, cytosine, guanine, and thymine with abbreviations ```A```, ```C```, ```G```, and ```T```. Given a *DNA sequence* e.g. ```ACGATGAGGCTCAT```, one can reverse engineer (with negligible loss of information) the corresponding DNA molecule.

Parts of a DNA molecule can *transcribe* into an RNA molecule. In this process, thymine gets replaced by uracil (```U```). 


1. Write a function ```dna_to_rna``` to convert a given DNA sequence $s$ into an RNA sequence. For the sake of exercise, use ```dict()``` to store the symbol to symbol encoding rules. Create a program to test your function.

In [1]:

def dna_to_rna(s: str):
    mapping = dict(zip("ACGT","ACGU"))
    return ''.join(mapping[base] for base in s)
    # below not used because the exercise asked for the dict method
    # return s.replace("T", "U") 
    
if __name__ == '__main__':
    print(dna_to_rna("AACGTGATTTC"))

AACGUGAUUUC


### Idea of solution
Makes a dictionary where DNA bases are keys and RNA bases are values. Then returns the corresponding RNA sequence by substituting with the dictionary.

## Proteins

Like DNA and RNA, protein molecule can be interpreted as a chain of smaller molecules, where the bases are now amino acids. RNA molecule may *translate* into a protein molecule, but instead of base by base, three bases of RNA correspond to one base of protein. That is, RNA sequence is read triplet (called codon) at a time. 

2. Consider the codon to amino acid conversion table in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html. Write a function ```get_dict``` to read the table into a ```dict()```, such that for each RNA sequence of length 3, say $\texttt{AGU}$, the hash table stores the conversion rule to the corresponding amino acid. You may store the html page to your local src directory,
and parse that file.

In [9]:
def get_dict():
    df = pd.read_csv("src/codon_conversion_table.csv", sep=" ")
    triplet = df["triplet"]
    amino_acid = df["amino acid"]
    return dict(zip(triplet, amino_acid))
    
if __name__ == '__main__':
    codon_to_aa = get_dict()
    print(codon_to_aa)

{'UUU': 'F', 'UCU': 'S', 'UAU': 'Y', 'UGU': 'C', 'UUC': 'F', 'UCC': 'S', 'UAC': 'Y', 'UGC': 'C', 'UUA': 'L', 'UCA': 'S', 'UAA': '*', 'UGA': '*', 'UUG': 'L', 'UCG': 'S', 'UAG': '*', 'UGG': 'W', 'CUU': 'L', 'CCU': 'P', 'CAU': 'H', 'CGU': 'R', 'CUC': 'L', 'CCC': 'P', 'CAC': 'H', 'CGC': 'R', 'CUA': 'L', 'CCA': 'P', 'CAA': 'Q', 'CGA': 'R', 'CUG': 'L', 'CCG': 'P', 'CAG': 'Q', 'CGG': 'R', 'AUU': 'I', 'ACU': 'T', 'AAU': 'N', 'AGU': 'S', 'AUC': 'I', 'ACC': 'T', 'AAC': 'N', 'AGC': 'S', 'AUA': 'I', 'ACA': 'T', 'AAA': 'K', 'AGA': 'R', 'AUG': 'M', 'ACG': 'T', 'AAG': 'K', 'AGG': 'R', 'GUU': 'V', 'GCU': 'A', 'GAU': 'D', 'GGU': 'G', 'GUC': 'V', 'GCC': 'A', 'GAC': 'D', 'GGC': 'G', 'GUA': 'V', 'GCA': 'A', 'GAA': 'E', 'GGA': 'G', 'GUG': 'V', 'GCG': 'A', 'GAG': 'E', 'GGG': 'G'}


### Idea of solution

Reads a file called codon_conversion_table where all the data is arranged in a csv file. Loads this into a dataframe, and uses the columns "triplet" and "amino acid" to form a dictionary where codons are keys and proteins are values. 

3. Use the same conversion table as above, but now write function `get_dict_list` to read the table into a `dict()`, such that for each amino acid the hash table stores the list of codons encoding it.    

In [10]:
def get_dict_list():
    forward = get_dict()
    output = {}
    for key, value in forward.items():
        if value in output.keys():
            output[value].append(key)
        else:
            output[value] = [key]
    return output
        
    
if __name__ == '__main__':
    aa_to_codons = get_dict_list()
    print(aa_to_codons)

{'F': ['UUU', 'UUC'], 'S': ['UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'], 'Y': ['UAU', 'UAC'], 'C': ['UGU', 'UGC'], 'L': ['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'], '*': ['UAA', 'UGA', 'UAG'], 'W': ['UGG'], 'P': ['CCU', 'CCC', 'CCA', 'CCG'], 'H': ['CAU', 'CAC'], 'R': ['CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], 'Q': ['CAA', 'CAG'], 'I': ['AUU', 'AUC', 'AUA'], 'T': ['ACU', 'ACC', 'ACA', 'ACG'], 'N': ['AAU', 'AAC'], 'K': ['AAA', 'AAG'], 'M': ['AUG'], 'V': ['GUU', 'GUC', 'GUA', 'GUG'], 'A': ['GCU', 'GCC', 'GCA', 'GCG'], 'D': ['GAU', 'GAC'], 'G': ['GGU', 'GGC', 'GGA', 'GGG'], 'E': ['GAA', 'GAG']}


### Idea of solution

Loops through the {Triplet: Protein} dictionary and forms a new dictionary {Protein: [Triplets]}. In the loop, if a key "New_protein" is not yet present, adds it in with value as [accompanying codon]. If the key is present, then just appends the codon to the corresponding list. 

With the conversion tables at hand, the following should be trivial to solve.

4. Fill in function ```rna_to_prot``` in the stub solution to convert a given DNA sequence $s$ into a protein sequence. 
You may use the dictionaries from exercises 2 and 3. You can test your program with `ATGATATCATCGACGATGTAG`.

In [11]:

def rna_to_prot(s: str):
    triplets = [s[i:i+3] for i in range(0,len(s), 3)]
    output = ""
    conversion_table = get_dict()
    for codon in triplets:
        output += conversion_table[codon]
    return output

def dna_to_prot(s):
    return rna_to_prot(dna_to_rna(s))

if __name__ == '__main__':
    print(dna_to_prot("ATGATATCATCGACGATGTAG"))

MISSTM*


### Idea of solution

Splits up the string s into codons and places the codons in a list. Then loops through codons and appends the corresponding protein to an output string. Returns the result.

You may notice that there are $4^3=64$ different codons, but only 20 amino acids. That is, some triplets encode the same amino acid.  

## Reverse translation

It has been observed that among the codons coding the same amino acid, some are more frequent than others. These frequencies can be converted to probabilities. E.g. consider codons `AUU`, `AUC`, and `AUA` that code for amino acid isoleucine.
If they are observed, say, 36, 47, 17 times, respectively, to code isoleucine in a dataset, the probability that a random such event is `AUU` $\to$ isoleucine is 36/100.

This phenomenon is called *codon adaptation*, and for our purposes it works as a good introduction to generation of random sequences under constraints.   

5. Consider the codon adaptation frequencies in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html and read them into a ```dict()```, such that for each RNA sequence of length 3, say `AGU`, the hash table stores the probability of that codon among codons encoding the same amino acid.
Put your solution in the ```get_probabability_dict``` function. Use the column "([number])" to estimate the probabilities, as the two preceding columns contain truncated values.  

In [13]:

def get_probabability_dict():
    df = pd.read_csv("src/codon_conversion_table.csv", sep=" ")
    df.set_index("triplet", inplace=True)
    
    #makes a dictionary of all amino acids as keys, and number of times the amino acid has appeared as values
    amino_acid_sums = dict.fromkeys(np.unique(df["amino acid"]))
    df["(number)"] = df["(number)"].str.strip("()").astype(int)

    for acid in amino_acid_sums.keys():
        mask = df["amino acid"] == acid
        df2 = df[mask]
        amino_acid_sums[acid] = df2["(number)"].sum()
    
    #finds probability by codon
    codons = dict.fromkeys(df.index)
    for codon in codons:
        num = df.loc[codon,"(number)"]
        den = amino_acid_sums[df.loc[codon,"amino acid"]]
        codons[codon] = num/den
        
    return codons
    
if __name__ == '__main__':
    codon_to_prob = get_probabability_dict()

    items = sorted(codon_to_prob.items(), key=lambda x: x[0])
    for i in range(1 + len(items)//6):
        print("\t".join(
            f"{k}: {v:.6f}"
            for k, v in items[i*6:6+i*6]
        ))

AAA: 0.434049	AAC: 0.529633	AAG: 0.565951	AAU: 0.470367	ACA: 0.284188	ACC: 0.355232
ACG: 0.113812	ACU: 0.246769	AGA: 0.214658	AGC: 0.239938	AGG: 0.211091	AGU: 0.149602
AUA: 0.169062	AUC: 0.469866	AUG: 1.000000	AUU: 0.361072	CAA: 0.265017	CAC: 0.581485
CAG: 0.734983	CAU: 0.418515	CCA: 0.276603	CCC: 0.323470	CCG: 0.113196	CCU: 0.286731
CGA: 0.108812	CGC: 0.183777	CGG: 0.201554	CGU: 0.080108	CUA: 0.071380	CUC: 0.195577
CUG: 0.395702	CUU: 0.131716	GAA: 0.422453	GAC: 0.535458	GAG: 0.577547	GAU: 0.464542
GCA: 0.228121	GCC: 0.399781	GCG: 0.106176	GCU: 0.265922	GGA: 0.249922	GGC: 0.337109
GGG: 0.249882	GGU: 0.163087	GUA: 0.116577	GUC: 0.238306	GUG: 0.463346	GUU: 0.181770
UAA: 0.297019	UAC: 0.556662	UAG: 0.236738	UAU: 0.443338	UCA: 0.150517	UCC: 0.217960
UCG: 0.054398	UCU: 0.187586	UGA: 0.466243	UGC: 0.543843	UGG: 1.000000	UGU: 0.456157
UUA: 0.076568	UUC: 0.535866	UUG: 0.129058	UUU: 0.464134


### Idea of solution

The numbers column contains the number of each codon that appeared. First, the code removes all formatting from this column and turns it into a column of integers. Then, it takes each amino acid and sums up the number of appearances of all codons that code for that particular amino acid and stores this in a dictionary in the form

amino acid: [amino acid appearances].

Then, another dictionary is created with codons as keys. The value for each codon is found by codon count/amino acid appearances of the associated amino acid

$\frac{number[codon]}{sum[amino acids]}$

Now you should have everything in place to easily solve the following.


6. Write a class ```ProteinToMaxRNA``` with a ```convert``` method which converts a protein sequence into the most likely RNA sequence to be the source of this protein. Run your program with `LTPIQNRA`.

In [14]:
class ProteinToMaxRNA:
    
    def __init__(self):
        pass

    def convert(self, s: str):
        output = ""
        dict_list = get_dict_list()
        probability_list = get_probabability_dict()
        for protein in s:
            options = dict_list[protein]
            highest_prob = 0
            probable_amino = ""
            for option in options:
                if probability_list[option] > highest_prob:
                    probable_amino = option
                    highest_prob = probability_list[option]
            output += probable_amino
            
        return output


if __name__ == '__main__':
    protein_to_rna = ProteinToMaxRNA()
    print(protein_to_rna.convert("LTPIQNRA"))

CUGACCCCCAUCCAGAACAGAGCC


### Idea of solution

1. Goes through each character (i.e. amino acid) in string s. Checks from the probability dictionary which codon is most likely to produce the amino acid.
2. Appends the codon to an output string. The process is repeated for all amino acids. When this is done, the output string is returned by the function.

Now we are almost ready to produce random RNA sequences that code a given protein sequence. For this, we need a subroutine to *sample from a probability distribution*. Consider our earlier example of probabilities 36/100, 47/100, and 17/100 for `AUU`, `AUC`, and `AUA`, respectively. 
Let us assume we have a random number generator ```random()``` that returns a random number from interval $[0,1)$. We may then partition the unit interval according to cumulative probabilities to $[0,36/100), [36/100,83/100), [83/100,1)$, respectively. Depending which interval the number ```random()``` hits, we select the codon accordingly.

7. Write a function ```random_event``` that chooses a random event, given a probability distribution (set of events whose probabilities sum to 1).
You can use function ```random.uniform``` to produce values uniformly at random from the range $[0,1)$. The distribution should be given to your function as a dictionary from events to their probabilities.

In [16]:
def random_event(dist):
    """
    Takes as input a dictionary from events to their probabilities.
    Return a random event sampled according to the given distribution.
    The probabilities must sum to 1.0
    """
    r = random.uniform(0,1)
    cumulative = 0
    for base, prob in dist.items():
        cumulative += prob
        if r <= cumulative:
            return base 


    return next(iter(dist))

if __name__ == '__main__':
    distribution = dict(zip("ACGT", [0.10, 0.35, 0.15, 0.40]))
    print(", ".join(random_event(distribution) for _ in range(29)))

C, A, G, T, T, T, T, C, C, T, C, C, C, T, A, C, C, T, T, T, C, T, T, G, T, T, G, C, C


### Idea of solution

1. Generates a random number between zero and one. 
2. Loops through each item in the dictionary. If the random number is less than the higher limit of (P(event) +  P(all other events before it)), move on to the next event. If it is lower than the higher limit, then return the event. 

With this general routine, the following should be easy to solve.
 
8. Write a class ```ProteinToRandomRNA``` to produce a random RNA sequence encoding the input protein sequence according to the input codon adaptation probabilities. The actual conversion is done through the ```convert``` method. Run your program with `LTPIQNRA`.

In [17]:
class ProteinToRandomRNA(object):
    
    def __init__(self):
        pass

    def convert(self, s):
        dict_list = get_dict_list()
        probabiliy_dict = get_probabability_dict()
        output = ""
        for acid in s:
            codons = dict_list[acid]
            probability = [probabiliy_dict[codon] for codon in codons]
            output += random_event(dict(zip(codons, probability)))
        return output

        
if __name__ == '__main__':
    protein_to_random_codons = ProteinToRandomRNA()
    print(protein_to_random_codons.convert("LTPIQNRA"))

UUGACUCCUAUUCAGAACAGAGCA


### Idea of solution


sequence -> amino acid -> corresponding codons -> codon & codon probabilities passed to convert()

1. Loops through all the amino acids in the sequence.
2. In one loop, it finds all the associated codons of an amino acid and the probability with which they appear and form this into a dictionary. Using the random_event function, chooses a random item from this dictionary with weighted probability.  The weight is determined by the probability dictionary.

## Generating DNA sequences with higher-order Markov chains

We will now reuse the machinery derived above in a related context. We go back to DNA sequences, and consider some easy statistics that can be used to characterize the sequences. 
First, just the frequencies of bases $\texttt{A}$, $\texttt{C}$, $\texttt{G}$, $\texttt{T}$ may reveal the species from which the input DNA originates; each species has a different base composition that has been formed during evolution. 
More interestingly, the areas where DNA to RNA transcription takes place (coding region) have an excess of $\texttt{C}$ and $\texttt{G}$ over $\texttt{A}$ and $\texttt{T}$. To detect such areas a common routine is to just use a *sliding window* of fixed size, say $k$, and compute for each window position 
$T[i..i+k-1]$ the base frequencies, where $T[1..n]$ is the input DNA sequence. When sliding the window from  $T[i..i+k-1]$ to $T[i+1..i+k]$ frequency $f(T[i])$ gets decreases by one and $f(T[i+k])$ gets increased by one. 

9. Write a *generator* ```sliding_window``` to compute sliding window base frequencies so that each moving of the window takes constant time. We saw in the beginning of the course one way how to create generators using
  generator expression. Here we use a different way. For the function ```sliding_window``` to be a generator, it must have at least   one ```yield``` expression, see [https://docs.python.org/3/reference/expressions.html#yieldexpr](https://docs.python.org/3/reference/expressions.html#yieldexpr).
  
  Here is an example of a generator expression that works similarily to the built in `range` generator:
  ```Python
  def range(a, b=None, c=1):
      current = 0 if b == None else a
      end = a if b == None else b
      while current < end:
          yield current
          current += c
  ```
  A yield expression can be used to return a value and *temporarily* return from the function.

In [18]:
def sliding_window(s, k):
    """
    This function returns a generator that can be iterated over all
    starting position of a k-window in the sequence.
    For each starting position the generator returns the nucleotide frequencies
    in the window as a dictionary.
    """
    for i in range(k,len(s)+1):
        output = {"A":0,"C":0,"G":0,"T":0}
        for char in s[i-k:i]:
            output[char] += 1
        yield output
        
    
if __name__ == '__main__':
    s = "TCCCGACGGCCTTGCC"
    for d in sliding_window(s, 4):
        print(d)

{'A': 0, 'C': 3, 'G': 0, 'T': 1}
{'A': 0, 'C': 3, 'G': 1, 'T': 0}
{'A': 1, 'C': 2, 'G': 1, 'T': 0}
{'A': 1, 'C': 2, 'G': 1, 'T': 0}
{'A': 1, 'C': 1, 'G': 2, 'T': 0}
{'A': 1, 'C': 1, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 1, 'T': 1}
{'A': 0, 'C': 2, 'G': 0, 'T': 2}
{'A': 0, 'C': 1, 'G': 1, 'T': 2}
{'A': 0, 'C': 1, 'G': 1, 'T': 2}
{'A': 0, 'C': 2, 'G': 1, 'T': 1}


### Idea of solution

1. Initializes a dictionary with each base set to count 0. Yields count of each character in a certain length of character, and moves rightwards by one position for the next time.

 
Our models so far have been so-called *zero-order* models, as each event has been independent of other events. With sequences, the dependencies of events are naturally encoded by their *contexts*. Considering that a sequence is produced from left-to-right, a *first-order* context for $T[i]$ is $T[i-1]$, that is, the immediately preceding symbol. *First-order Markov chain* is a sequence produced by generating $c=T[i]$ with the probability of event of seeing symbol $c$ after previously generated symbol $a=T[i-1]$. The first symbol of the chain is sampled according to the zero-order model.  
The first-order model can naturally be extended to contexts of length $k$, with $T[i]$ depending on $T[i-k..i-1]$. Then the first $k$ symbols of the chain are sampled according to the zero-order model.  The following assignments develop the routines to work with the *higher-order Markov chains*. 
In what follows, a $k$-mer is a substring $T[i..i+k-1]$ of the sequence at an arbitrary position. 

10. Write function ```context_list``` that given an input DNA sequence $T$ associates to each $k$-mer $W$ the concatenation of all symbols $c$ that appear after context $W$ in $T$, that is, $T[i..i+k]=Wc$. For example, <span style="color:red; font:courier;">GA</span> is associated to <span style="color:blue; font: courier;">TCT</span> in $T$=<span style="font: courier;">AT<span style="color:red;">GA</span><span style="color:blue;">T</span>ATCATC<span style="color:red;">GA</span><span style="color:blue;">C</span><span style="color:red;">GA</span><span style="color:blue;">T</span>GTAG</span>, when $k=2$.

In [19]:
def context_list(s, k):
    output = {}
    for i in range(len(s)-k):
        k_mer = s[i:i+k]
        dependent = s[i+k]
        if k_mer in output.keys():
            output[k_mer] += dependent
        else:
            output[k_mer] = dependent
    return output
    
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATCTAG"
    d = context_list(s, k)
    print(d)

{'AT': 'GACCC', 'TG': 'A', 'GA': 'TCT', 'TA': 'TG', 'TC': 'AGT', 'CA': 'T', 'CG': 'AA', 'AC': 'G', 'CT': 'A'}


### Idea of solution

Loops through the string, taking a k-length substring. Every loop moves the substring to the right by one character. The k-length substring is called a k-mer, and the character after the k-mer that is stored in a variable called dependent.

If the k-mer is in the output dictionary, then the dependent is appended to the value of the k-mer. Otherwise, a new key-value pair is formed with the k-mer as the key and the dependent as the value.

This dictionary is then returned.


11. With the above solution, write function ```context_probabilities``` to count the frequencies of symbols in each context and convert these frequencies into probabilities. Run `context_probabilities` with $T=$ `ATGATATCATCGACGATGTAG` and $k$ values 0 and 2.

In [20]:
def context_probabilities(s, k):
    processed_list = context_list(s,k)
    output = {}
    for k_mer, bases in processed_list.items():
        new_value = {}
        unique_bases = set(bases)
        for base in unique_bases:
            new_value[base] = bases.count(base)/len(bases)
        output[k_mer] = new_value
    return output

    
if __name__ == '__main__':
    print(context_probabilities("ATGATATCATCGACGATGTAG", 2))
    print(context_probabilities("ATGATATCATCGACGATGTAG", 0))


{'AT': {'C': 0.4, 'A': 0.2, 'G': 0.4}, 'TG': {'T': 0.5, 'A': 0.5}, 'GA': {'T': 0.6666666666666666, 'C': 0.3333333333333333}, 'TA': {'T': 0.5, 'G': 0.5}, 'TC': {'A': 0.5, 'G': 0.5}, 'CA': {'T': 1.0}, 'CG': {'A': 1.0}, 'AC': {'G': 1.0}, 'GT': {'A': 1.0}}
{'': {'T': 0.2857142857142857, 'C': 0.14285714285714285, 'A': 0.3333333333333333, 'G': 0.23809523809523808}}


### Idea of solution

1. Uses context list to get all characters that follow k-mers. A empty dictionary is initialized

2. Loops through all the k-mer:character pairs. In one loop, it loops through all the unique characters found after the k-mer and finds the frequency of that character occuring by $\frac{count(character)}{count(all characters following the k-mer)}$.

3. This is added to the empty dictionary in the form k-mer: {character1: probability of appearing ...}

12. With the above solution and the function ```random_event``` from the earlier exercise, write class ```MarkovChain```. Its ```generate``` method should generate a random DNA sequence following the original $k$-th order Markov chain probabilities. 

In [21]:
class MarkovChain:
    
    def __init__(self, zeroth, kth, k=2):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def generate(self, n, seed=None):
        random.seed(seed)
        output = ""
        output = "".join(random_event(self.zeroth) for _ in range(min(self.k,n)))
        new_k = self.k
        for i in range(self.k,n):
            if output[new_k - self.k:new_k] in self.kth.keys():
                output += random_event(self.kth[output[new_k - self.k:new_k]])
            else:
                output += random_event(self.zeroth)
            new_k+=1
            
        return output

if __name__ == '__main__':
    zeroth = {'A': 0.2, 'C': 0.19, 'T': 0.31, 'G': 0.3}
    kth = {'GT': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0},
           'CA': {'A': 0.0, 'C': 0.0, 'T': 1.0, 'G': 0.0},
           'TC': {'A': 0.5, 'C': 0.0, 'T': 0.0, 'G': 0.5},
           'GA': {'A': 0.0, 'C': 0.3333333333333333, 'T': 0.6666666666666666, 'G': 0.0},
           'TG': {'A': 0.5, 'C': 0.0, 'T': 0.5, 'G': 0.0},
           'AT': {'A': 0.2, 'C': 0.4, 'T': 0.0, 'G': 0.4},
           'TA': {'A': 0.0, 'C': 0.0, 'T': 0.5, 'G': 0.5},
           'AC': {'A': 0.0, 'C': 0.0, 'T': 0.0, 'G': 1.0},
           'CG': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0}}
    n = 4
    mc = MarkovChain(zeroth, kth)
    print(mc.generate(n))

CTTC


### Idea of solution

check if last k-th letters in kth. If yes, generate from this. Otherwise, generate from zeroth.

If you have survived so far without problems, please run your program a few more times with different inputs. At some point you should get a lookup error in your hash-table! The reason for this is not your code, but the way we defined the model: Some $k$-mers may not be among the training data (input sequence $T$), but such can be generated as the first $k$-mer that is generated using the zero-order model.  

A general approach to fixing such issues with incomplete training data is to use *pseudo counts*. That is, all imaginable events are initialized to frequency count 1.   

13. Write a new solution `context_pseudo_probabilities` based on the solution to problem 11. But this time use pseudo counts in order to obtain a $k$-th order Markov chain that can assign a probability for any DNA sequence. You may use the standard library function `itertools.product` to iterate over all $k$-mer of given length (`product("ACGT", repeat=k)`).

In [22]:
def context_pseudo_probabilities(s, k):
    processed_list = context_list(s,k)
    for item in product("ACGT", repeat=k):
        comb = "".join(item)
        if comb in processed_list.keys():
            processed_list[comb] += "ACGT"
        else:
            processed_list[comb] = "ACGT"

    output = {}
    for k_mer, bases in processed_list.items():
        new_value = {}
        unique_bases = set(bases)
        for base in unique_bases:
            new_value[base] = bases.count(base)/len(bases)
        output[k_mer] = new_value
    return output


if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    kth = context_pseudo_probabilities(s, k)
    zeroth = context_pseudo_probabilities(s, 0)[""]
    print(f"zeroth: {zeroth}")
    print("\n".join(f"{k}: {dict(v)}" for k, v in kth.items()))
    
    print("\n", MarkovChain(zeroth, kth, k).generate(20))

zeroth: {'T': 0.28, 'C': 0.16, 'A': 0.32, 'G': 0.24}
AT: {'T': 0.1111111111111111, 'C': 0.3333333333333333, 'A': 0.2222222222222222, 'G': 0.3333333333333333}
TG: {'T': 0.3333333333333333, 'C': 0.16666666666666666, 'A': 0.3333333333333333, 'G': 0.16666666666666666}
GA: {'T': 0.42857142857142855, 'C': 0.2857142857142857, 'A': 0.14285714285714285, 'G': 0.14285714285714285}
TA: {'T': 0.3333333333333333, 'C': 0.16666666666666666, 'A': 0.16666666666666666, 'G': 0.3333333333333333}
TC: {'T': 0.16666666666666666, 'C': 0.16666666666666666, 'A': 0.3333333333333333, 'G': 0.3333333333333333}
CA: {'T': 0.4, 'C': 0.2, 'A': 0.2, 'G': 0.2}
CG: {'T': 0.16666666666666666, 'C': 0.16666666666666666, 'A': 0.5, 'G': 0.16666666666666666}
AC: {'T': 0.2, 'C': 0.2, 'A': 0.2, 'G': 0.4}
GT: {'T': 0.2, 'C': 0.2, 'A': 0.4, 'G': 0.2}
AA: {'T': 0.25, 'C': 0.25, 'A': 0.25, 'G': 0.25}
AG: {'T': 0.25, 'C': 0.25, 'A': 0.25, 'G': 0.25}
CC: {'T': 0.25, 'C': 0.25, 'A': 0.25, 'G': 0.25}
CT: {'T': 0.25, 'C': 0.25, 'A': 0.25, 

### Idea of solution

every single possiblility has ACGT after it. Then, sequences that are present in S have letters appended to it, and then the probabilities are calculated again.

14. Write class ```MarkovProb``` that given the $k$-th order Markov chain developed above to the constructor, its method ```probability``` computes the probability of a given input DNA sequence.

In [23]:
class MarkovProb:
    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def probability(self, s):
        prob = 1
        for i in range(min(len(s),self.k)):
            prob *= self.zeroth[s[i]]
        if self.k > len(s):
            return prob
        for i in range(len(s)-self.k):
            k_mer = s[i:i+self.k]
            dependent = s[i+self.k]
            prob *= self.kth[k_mer][dependent]
        return prob

    
if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovProb(k, zeroth, kth)
    s="ATGATA"
    print(f"Probability of sequence {s} is {mc.probability(s)}")

Probability of sequence ATGATA is 0.0009481481481481482


### Idea of solution

Find probability of first k letters occurring, just by multiplying them together.

next, take k letters and find the probability of the next one occuring. multiply this and continue

With the last assignment you might end up in trouble with precision, as multiplying many small probabilities gives a really small number in the end. There is an easy fix by using so-called log-transform. 
Consider computation of $P=s_1 s_2 \cdots s_n$, where $0\leq s_i\leq 1$ for each $i$. Taking logarithm in base 2 from both sides gives $\log _2 P= \log _2 (s_1 s_2 \cdots s_n)=\log_2 s_1 + \log_2 s_2 + \cdots \log s_n= \sum_{i=1}^n \log s_i$, with repeated application of the property that the logarithm of a multiplication of two numbers is the sum of logarithms of the two numbers taken separately. The results is abbreviated as log-probability.

15. Write class ```MarkovLog``` that given the $k$-th order Markov chain developed above to the constructor, its method ```log_probability``` computes the log-probability of a given input DNA sequence. Run your program with $T=$ `ATGATATCATCGACGATGTAG` and $k=2$.

In [24]:
class MarkovLog(object):

    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth

    def log_probability(self, s):
        prob = 1
        for i in range(min(len(s),self.k)):
            prob *= self.zeroth[s[i]]
        if self.k > len(s):
            return prob
        for i in range(len(s)-self.k):
            k_mer = s[i:i+self.k]
            dependent = s[i+self.k]
            prob *= self.kth[k_mer][dependent]
        return np.log2(prob)

    
if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovLog(2, zeroth, kth)
    s="ATGATATCATCGACGATGTAG"
    print(f"Log probability of sequence {s} is {mc.log_probability(s)}")

Log probability of sequence ATGATATCATCGACGATGTAG is -31.717831515538307


### Idea of solution

Same as previous, just with some log2 sprinkled on top woweeee

Finally, if you try to use the code so far for very large inputs, you might observe that the concatenation of symbols following a context occupy considerable amount of space. This is unnecessary, as we only need the frequencies. 

16. Optimize the space requirement of your code from exercise 13 for the $k$-th order Markov chain by replacing the concatenations by direct computations of the frequencies. Implement this as the
  ```better_context_probabilities``` function.

In [25]:
def better_context_probabilities(s, k):
    output = {}
    for item in product("ACGT", repeat=k):
        output["".join(item)] = dict(zip("ACGT",[1]*4))
    for i in range(len(s)-k):
        k_mer = s[i:i+k]
        dependent = s[i+k]
        output[k_mer][dependent] += 1
    for context in output:
        total = sum(output[context].values())
        for frequency in output[context]:
            output[context][frequency] = output[context][frequency]/total
    return output

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    d = better_context_probabilities(s, k)
    print("\n".join(f"{k}: {v}" for k, v in d.items()))

AA: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AC: {'A': 0.2, 'C': 0.2, 'G': 0.4, 'T': 0.2}
AG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AT: {'A': 0.2222222222222222, 'C': 0.3333333333333333, 'G': 0.3333333333333333, 'T': 0.1111111111111111}
CA: {'A': 0.2, 'C': 0.2, 'G': 0.2, 'T': 0.4}
CC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CG: {'A': 0.5, 'C': 0.16666666666666666, 'G': 0.16666666666666666, 'T': 0.16666666666666666}
CT: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GA: {'A': 0.14285714285714285, 'C': 0.2857142857142857, 'G': 0.14285714285714285, 'T': 0.42857142857142855}
GC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GT: {'A': 0.4, 'C': 0.2, 'G': 0.2, 'T': 0.2}
TA: {'A': 0.16666666666666666, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.3333333333333333}
TC: {'A': 0.3333333333333333, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.16666666666666666}
TG: {'A': 0.3333333333333333, 'C': 0.16666666666666

### Idea of solution

fill in

Initialize all possibilites with 1. Then add based on the next base,

While the earlier approach of explicit concatenation of symbols following a context suffered from inefficient use of space, it does have a benefit of giving another much simpler strategy to sample from the distribution: 
observe that an element of the concatenation taken uniformly randomly is sampled exactly with the correct probability. 

17. Revisit the solution 12 and modify it to directly sample from the concatenation of symbols following a context. The function ```np.random.choice``` may be convenient here. Implement the modified version as the new `SimpleMarkovChain` class.

In [ ]:
import re

class SimpleMarkovChain(object):
    def __init__(self, s, k):
        self.s = s
        self.k = k
        self.s_as_list = [char for char in s]

    def generate(self, n, seed=None):
        random.seed(seed)
        k = self.k
        output = ""
        
        if n == 0:
            return output

        output = "".join(random.choice(self.s_as_list) for _ in range(min(n,k)))

        for i in range(k,n):
            pattern = re.escape(output[i-k:i]) + "."
            occurances = re.findall(pattern,self.s)
            if occurances:
                output += random.choice(occurances)[-1]
            else:
                output += random.choice(self.s_as_list)

        return output

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    n = 8
    seed = 444
    mc = SimpleMarkovChain(s, k)
    k=2
    s="ATGATATCATCGACGATGTAG"
    seed=0
    mc = SimpleMarkovChain(s, k)
    s1 = mc.generate(40, seed)
    s2 = mc.generate(40, seed)

TTAGACGA
TTAGACGA


### Idea of solution

Uses regex to find any substring with the desired k-mer + any random thing after. If this exists, chooses one of the random things after and appends it to output.

Else it just takes a random base.

## $k$-mer index

Our $k$-th order Markov chain can now be modified to a handy index structure called $k$-mer index. This index structure associates to each $k$-mer its list of occurrence positions in DNA sequence $T$.  Given a query $k$-mer $W$, one can thus easily list all positions $i$ with  $T[i..k-1]=W$.

18. Implement function ```kmer_index``` inspired by your earlier code for the $k$-th order Markov chain. Test your program with `ATGATATCATCGACGATGTAG` and $k=2$.

In [27]:
def kmer_index(s, k):
    positions = {}
    for i in range(len(s)-k+1):
        k_mer = s[i:i+k]
        if k_mer in positions:
            positions[k_mer].append(i)
        else:
            positions[k_mer] = [i]

    return positions

if __name__ == '__main__':
    k=2
    s = "ATGATATCATCGACGATGTAG"
    print("Using string:")
    print(s)
    print("".join([str(i%10) for i in range(len(s))]))
    print(f"\n{k}-mer index is:")
    d=kmer_index(s, k)
    print(dict(d))

Using string:
ATGATATCATCGACGATGTAG
012345678901234567890

2-mer index is:
{'AT': [0, 3, 5, 8, 15], 'TG': [1, 16], 'GA': [2, 11, 14], 'TA': [4, 18], 'TC': [6, 9], 'CA': [7], 'CG': [10, 13], 'AC': [12], 'GT': [17], 'AG': [19]}


## Comparison of probability distributions

Now that we know how to learn probability distributions from data, we might want to compare two such distributions, for example, to test if our programs work as intended. 

Let $P=\{p_1,p_2,\ldots, p_n\}$ and $Q=\{q_1,q_2,\ldots, q_n\}$ be two probability distributions for the same set of $n$ events. This means $\sum_{i=1}^n p_i=\sum_{i=1}^n q_i=1$, $0\leq p_j \leq 1$, and $0\leq q_j \leq 1$ for each event $j$. 

*Kullback-Leibler divergence* is a measure $d()$ for the *relative entropy* of $P$ with respect to $Q$ defined as 
$d(P||Q)=\sum_{i=1}^n p_i \log\frac{p_i}{q_i}$.


This measure is always non-negative, and 0 only when $P=Q$. It can be interpreted as the gain of knowing $Q$ to encode $P$. Note that this measure is not symmetric.

19. Write function ```kullback_leibler``` to compute $d(P||Q)$. Test your solution by generating a random RNA sequence
  encoding the input protein sequence according to the input codon adaptation probabilities.
  Then you should learn the codon adaptation probabilities from the RNA sequence you generated.
  Then try the same with uniformly random RNA sequences (which don't have to encode any
  specific protein sequence). Compute the relative entropies between the
  three distribution (original, predicted, uniform) and you should observe a clear difference.
  Because $d(P||Q)$ is not symmetric, you can either print both $d(P||Q)$ and $d(Q||P)$,
  or their average.
  
  This problem may be fairly tricky. Only the `kullback_leibler` function is automatically tested. The codon probabilities is probably a useful helper function. The main guarded section can be completed by filling out the `pass` sections using tooling from previous parts and fixing the *placeholder* lines.

In [ ]:
def codon_probabilities(rna):
    """
    Given an RNA sequence, simply calculates the proability of
    all 3-mers empirically based on the sequence
    """
    protein_rna_pairs = get_dict()
    df = pd.DataFrame({"Protein":protein_rna_pairs.values(), "Appearances":0, "Frequency":0.0}, index=protein_rna_pairs.keys())
    for i in range(0,len(rna),3):
        df.loc[rna[i:i+3],"Appearances"] += 1

    for protein in np.unique(df["Protein"]):
        mask = df["Protein"]==protein
        protein_sum = np.sum(df[mask]["Appearances"])
        df.loc[mask,"Frequency"] = df.loc[mask,"Appearances"]/protein_sum
    
    return df["Frequency"].to_dict()


def kullback_leibler(p, q):
    """
    Computes Kullback-Leibler divergence between two distributions.
    Both p and q must be dictionaries from events to probabilities.
    The divergence is defined only when q[event] == 0 implies p[event] == 0.
    """

    final_sum = 0

    for codon in p.keys():
        p_i = p[codon]
        q_i = q[codon]
        if (p_i != 0) & (q_i == 0):
            raise ZeroDivisionError
        if p_i == 0:
            continue
        final_sum += p_i*np.log2(p_i/q_i)

    return final_sum

if __name__ == '__main__':
    aas = list("*ACDEFGHIKLMNPQRSTVWY") # List of amino acids
    n = 5000
    
    # generate a random :oice(aas, n))  
    protein_to_random_RNA = ProteinToRandomRNA()
    aas = np.random.choice(aas,n)
    assoc_rna = protein_to_random_RNA.convert(aas)
    
    # Maybe check that converting back to protein results in the same sequence
    check = rna_to_prot(assoc_rna)
    
    # Calculate codon probabilities of the rna sequence
    cp_predicted = codon_probabilities(assoc_rna)
    
    # Calculate codon probabilities based on the codon usage table
    cp_orig = get_probabability_dict()

    # Create a completely random RNA sequence and get the codon probabilities
    all_codons = list(get_dict().keys())
    completely_random = "".join(random.choice(all_codons) for i in range(n))
    cp_uniform = codon_probabilities(completely_random)

    print("d(original || predicted) =", kullback_leibler(cp_orig, cp_predicted))
    print("d(predicted || original) =", kullback_leibler(cp_predicted, cp_orig))
    print()
    print("d(original || uniform) =", kullback_leibler(cp_orig, cp_uniform))
    print("d(uniform || original) =", kullback_leibler(cp_uniform, cp_orig))
    print()
    print("d(predicted || uniform) =", kullback_leibler(cp_predicted, cp_uniform))
    print("d(uniform || predicted) =", kullback_leibler(cp_uniform, cp_predicted))

d(original || predicted) = 0.012551344119574204
d(predicted || original) = 0.0126327048179215

d(original || uniform) = 1.5132596547009656
d(uniform || original) = 1.6399902031839482

d(predicted || uniform) = 1.4874716171865092
d(uniform || predicted) = 1.6123827008080398


: 